In [1]:
import pandas as pd

In [2]:
col_names = ['season', 'team', 'games', 'shots',
             'goals', 'shot', 'asists', 'game_coef',
             'sa', 'ga', 'saves', 'inj','name']
statfile = pd.read_csv('raw_data/players_all_stats_75.csv', names = col_names)

In [3]:
statfile[2:10]

,season,team,games,shots,goals,shot,asists,game_coef,sa,ga,saves,inj,name
2,71.0,HC Jelgava Stars,3,2,0,0%,0,-2,0,0,-,0,NaN
3,69.0,HC Jelgava Stars,1,2,0,0%,0,-1,0,0,-,1,NaN
4,74.0,HC Jelgava Stars,29,50,4,8%,2,-11,0,0,-,3,NaN
5,72.0,HC Jelgava Stars,4,4,0,0%,0,-4,0,0,-,0,NaN
6,70.0,HC Jelgava Stars,5,1,0,0%,0,-2,0,0,-,1,NaN
7,75.0,HC Jelgava Stars,26,53,8,15.1%,8,5,0,0,-,0,Ernests Beļavskis
8,65.0,thunderforce2,0,0,0,-,0,0,0,0,-,0,NaN
9,75.0,Orlik Burton Opole,11,20,0,0%,2,2,0,0,-,0,NaN


## fill NaN values with names

In [4]:
df = statfile.fillna(method='ffill')

# Drop null games (season) , and foreign teams aswell

##### count only data when player was in my team

In [5]:
df = df[df['team'] == 'HC Jelgava Stars']
df =  df[df['games'] > 0]

In [9]:
df = df.reset_index(drop=True)
df[:7]

,season,team,games,shots,goals,shot,asists,game_coef,sa,ga,saves,inj,name
0,75.0,HC Jelgava Stars,26,46,3,6.5%,4,-5,0,0,-,0,Vladimirs Znotiņš
1,73.0,HC Jelgava Stars,35,44,4,9.1%,4,-2,0,0,-,3,Vladimirs Znotiņš
2,71.0,HC Jelgava Stars,3,2,0,0%,0,-2,0,0,-,0,Vladimirs Znotiņš
3,69.0,HC Jelgava Stars,1,2,0,0%,0,-1,0,0,-,1,Vladimirs Znotiņš
4,74.0,HC Jelgava Stars,29,50,4,8%,2,-11,0,0,-,3,Vladimirs Znotiņš
5,72.0,HC Jelgava Stars,4,4,0,0%,0,-4,0,0,-,0,Vladimirs Znotiņš
6,70.0,HC Jelgava Stars,5,1,0,0%,0,-2,0,0,-,1,Vladimirs Znotiņš


## group data by names and count sum for:
    -games
    -goals
    -asists
    -game_coef
    -inj

In [10]:
group = df.groupby('name')

In [12]:
final = group['games','goals','asists','game_coef','inj'].aggregate(sum).reset_index()

In [14]:
final[:3]

,name,games,goals,asists,game_coef,inj
0,Alberts Timanovskis,135,6,18,-29,15
1,Andris Kraulis,10,2,1,-4,3
2,Atis Zilgalvis,194,59,15,-18,1


#### add points column
    asists + goals

In [15]:
final['points'] = final['goals'] + final['asists']

In [17]:
final.sort_values(by=['points'],ascending=False).reset_index(drop=True)[:3]

,name,games,goals,asists,game_coef,inj,points
0,Dennis Ivarsson,210,22,174,71,9,196
1,Vasile Halip,300,109,61,-11,25,170
2,Maigonis Ušackis,277,94,37,8,23,131


### add column with number of total seasons played

In [18]:
seasons = group['season'].aggregate([min,max]).reset_index()

In [19]:
seasons['seasons'] = seasons['max'] - seasons['min'] + 1 

In [20]:
seasons[:2]

,name,min,max,seasons
0,Alberts Timanovskis,70.0,75.0,6.0
1,Andris Kraulis,72.0,75.0,4.0


In [21]:
final_table = pd.merge(final, seasons, on='name').sort_values(by=['points'],ascending=False).reset_index(drop=True)

In [22]:
table = final_table[['name','games','goals','asists','points','seasons']]

#### save data to csv

In [23]:
table.to_csv('clean_data/player_career_stats.csv')